In [1]:
import tensorflow.compat.v1 as tf

tf.compat.v1.disable_eager_execution()

# 상수 정의하기 --- (※1)
a = tf.constant(120, name="a")
b = tf.constant(130, name="b")
c = tf.constant(140, name="c")

# 변수 정의하기 --- (※2)
v = tf.Variable(0, name="v")

# 데이터 플로우 그래프 정의하기 --- (※3)
calc_op = a + b + c
assign_op = tf.assign(v, calc_op)

# 세션 실행하기 --- (※4)
sess = tf.Session()
sess.run(assign_op)

# v의 내용 출력하기 --- (※5)
print( sess.run(v) )

390


In [4]:
import tensorflow.compat.v1 as tf

tf.compat.v1.disable_eager_execution()

# 플레이스홀더 정의하기 --- (※1)
#a = tf.placeholder(tf.int32, [3]) # 정수 자료형 3개를 가진 배열
a = tf.placeholder(tf.int32, [None])

# 배열을 모든 값을 2배하는 연산 정의하기 --- (※2)
b = tf.constant(2)
x2_op = a * b

# 세션 시작하기 --- (※3)
sess = tf.Session()

# 플레이스홀더에 값을 넣고 실행하기 --- (※4)
r1 = sess.run(x2_op, feed_dict={ a:[1, 2, 3] })
print(r1)
r2 = sess.run(x2_op, feed_dict={ a:[10, 20, 10, 40] })
print(r2)

[2 4 6]
[20 40 20 80]


In [6]:
import pandas as pd
import numpy as np
import tensorflow.compat.v1 as tf

tf.compat.v1.disable_eager_execution()

# 키, 몸무게, 레이블이 적힌 CSV 파일 읽어 들이기 --- (※1)
csv = pd.read_csv("bmi.csv")

# 데이터 정규화 --- (※2)
csv["height"] = csv["height"] / 200
csv["weight"] = csv["weight"] / 100

# 레이블을 배열로 변환하기 --- (※3)
# - thin=(1,0,0) / normal=(0,1,0) / fat=(0,0,1)
bclass = {"thin": [1,0,0], "normal": [0,1,0], "fat": [0,0,1]}
csv["label_pat"] = csv["label"].apply(lambda x : np.array(bclass[x]))

# 테스트를 위한 데이터 분류 --- (※4)
test_csv = csv[15000:20000]
test_pat = test_csv[["weight","height"]]
test_ans = list(test_csv["label_pat"])

# 데이터 플로우 그래프 구축하기 --- (※5)
# 플레이스홀더 선언하기
x  = tf.placeholder(tf.float32, [None, 2]) # 키와 몸무게 데이터 넣기
y_ = tf.placeholder(tf.float32, [None, 3]) # 정답 레이블 넣기

# 변수 선언하기 --- (※6)
W = tf.Variable(tf.zeros([2, 3])); # 가중치
b = tf.Variable(tf.zeros([3])); # 바이어스

# 소프트맥스 회귀 정의하기 --- (※7)
y = tf.nn.softmax(tf.matmul(x, W) + b)

# 모델 훈련하기 --- (※8)
cross_entropy = -tf.reduce_sum(y_ * tf.log(y))
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(cross_entropy)

# 정답률 구하기
predict = tf.equal(tf.argmax(y, 1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(predict, tf.float32))

# 세션 시작하기
sess = tf.Session()
sess.run(tf.global_variables_initializer()) # 변수 초기화하기

# 학습시키기
for step in range(3500):
    i = (step * 100) % 14000
    rows = csv[1 + i : 1 + i + 100]
    x_pat = rows[["weight","height"]]
    y_ans = list(rows["label_pat"])
    fd = {x: x_pat, y_: y_ans}
    sess.run(train, feed_dict=fd)
    if step % 500 == 0:
        cre = sess.run(cross_entropy, feed_dict=fd)
        acc = sess.run(accuracy, feed_dict={x: test_pat, y_: test_ans})
        print("step=", step, "cre=", cre, "acc=", acc)

# 최종적인 정답률 구하기
acc = sess.run(accuracy, feed_dict={x: test_pat, y_: test_ans})
print("정답률 =", acc)

step= 0 cre= 108.66268 acc= 0.3242
step= 500 cre= 57.58866 acc= 0.8904
step= 1000 cre= 45.02092 acc= 0.898
step= 1500 cre= 41.65434 acc= 0.9566
step= 2000 cre= 34.664032 acc= 0.943
step= 2500 cre= 34.287033 acc= 0.9674
step= 3000 cre= 26.880764 acc= 0.9726
정답률 = 0.9712
